# Looking at our ASL files
The following notebooks takes a look at where the files are, and how information on them is recorded.
There are several irregularities we will have to code around in later analyses.

In [ ]:
import pydicom
from pydicom import dcmread
import sys
import numpy as np
import pandas as pd
import importlib
import ipywidgets as widgets
import matplotlib.pyplot as plt
import nibabel as nib
import glob
import os
import json
import shutil
sys.path.insert(0, '../')
from brickstudy import foundation


#fpath= 'BRICK-001/20231120/141946/MR/eASL_3_delays/1.2.826.0.1.3680043.10.104.1023698459453914814407809680136492834'

#ds=dcmread(fpath)

In [ ]:

json_files_we_want_real = glob.glob("/mnt/data/output/transfer_2217763_files_3b695c8d/**/MR/eASL_3_delays/*real.json", recursive=True)

In [ ]:

json_files_we_want_reala = glob.glob("/mnt/data/output/transfer_2217763_files_3b695c8d/**/MR/eASL_3_delays/*reala.json", recursive=True)

The first irregularity is that our files were not at the same level of a folder chain, we use the double wildcard ** to get around this

In [ ]:
json_files_we_want= json_files_we_want_real + json_files_we_want_reala
len(json_files_we_want)/2
#json_files_we_want

The second irregulairy is that the PCASL and CASL are not always file a for base file, they switch, so we need to keep track of which is which

In [ ]:

short_names = []
img_file_names = []
types = []
labeling_durations = []
delays = []
for file in json_files_we_want:
    #print(file)
    short_name = file[48:]
    #print(short_name)
    short_names.append(short_name)
    img_file_name = short_name[:-4] + 'nii.gz'
    img_file_names.append(img_file_name)
    with open(file) as fileo:
        loaded = json.load(fileo)
        ArterialSpinLabelingType = loaded['ArterialSpinLabelingType']
        delay = loaded['PostLabelingDelay']
        label_duration= loaded['LabelingDuration']
    types.append(ArterialSpinLabelingType)#ArterialSpinLabelingType
    delays.append(delay)
    labeling_durations.append(label_duration)
zipped = list(zip(short_names,types, img_file_names,delays, labeling_durations))
df = pd.DataFrame(zipped, columns= ['short_name_json', 'type', 'img','delay_legnth', 'label_duration'])
    #print(short_names)
#print(types)
pd.set_option('display.max_colwidth', None)
df.head(10)

In [ ]:
PCASL_files = df[df['type'] == 'PCASL']
print("We have", len(PCASL_files.delay_legnth.unique()), "type of label delay here, and the value/s are:",PCASL_files.delay_legnth.unique())
print("We have", len(PCASL_files.label_duration.unique()), "type of label duration here, and the value/s are:",PCASL_files.label_duration.unique())

Hallelluyah, at least all the actual PCASLimages are consistently delayed
Let's make lists of our PCASL files: 

In [ ]:
img_file = PCASL_files['img']
json_files =  PCASL_files['short_name_json']
json_files

Now let's make a note of the code that allows us to move just these files into a new folder we will calll

In [ ]:

long_names = []
img_file_names = []
types = []
labeling_durations = []
delays = []
for file in json_files_we_want:

    name = file
    long_names.append(name)
    img_file_name = name[:-4] + 'nii.gz'
    img_file_names.append(img_file_name)
    with open(file) as fileo:
        loaded = json.load(fileo)
        ArterialSpinLabelingType = loaded['ArterialSpinLabelingType']
        delay = loaded['PostLabelingDelay']
        label_duration= loaded['LabelingDuration']
    types.append(ArterialSpinLabelingType)#ArterialSpinLabelingType
    delays.append(delay)
    labeling_durations.append(label_duration)
zipped = list(zip(long_names,types, img_file_names,delays, labeling_durations))
df_long = pd.DataFrame(zipped, columns= ['long_name_json', 'type', 'img','delay_legnth', 'label_duration'])
    #print(short_names)
#print(types)
pd.set_option('display.max_colwidth', None)
df_long.head(3)

In [ ]:
PCASL_files = df_long[df_long['type'] == 'PCASL']
img_file = PCASL_files['img']
json_files =  PCASL_files['long_name_json']
#json_files

with open('/home/candace@mydre.org/json_holder_1/file1.txt','w') as outfile:
    for file in json_files:
        outfile.write(file)
        outfile.write('\n')


We wrote the files to a text file `/home/candace@mydre.org/json_holder_1/file1.txt`.
we will then have to use shell to copy the actual files

the command line was:

```sh
for f in $(< json_holder_1/file1.txt) ; do \
    dst=json_holder_1/$(echo $f | sed -e 's/.\+BRICK-\([0-9]\+\).\+/\1/').nii.gz; \
    src=$(echo $f | sed -e 's/[.]json/.nii.gz/'); \
    cp $src $dst; \
done
```

Backslashes are added to break lines (to make the command look more structured).
If it's difficult to copy and paste this command into the terminal, you can either:

1. In your terminal, type `export EDITOR=vi`, then press `C-x C-e`
   (While holding "Ctrl", press "x", release "x", press "e"). This will open `vi` editor,
   where you can edit the command whichever way you want, including removing the backslashes.
   Press `ESC :x` when done to close the editor and return to the shell.  Now the shell
   should display the command it will run.  Press `RET` to run the command.
2. Simply remove all the backslashes but then the enitre command needs to be on the single line.

Now we want to copy those over to the fsluser , where we can own everything, not be in root, and therefore run oxford_asl...
so again we must do some command line, and we will put them in a folder called pcasl_festival

i.e. sudo cp -R json_holder_1/ /home/fsluser/pcasl_festival

Then we want to wrie something to apply oxfor_asl in paralell - we have stored scripts for that on fsluser

Let open a raw data and perfusion map, first we will open the raw data:

In [ ]:

loaded = nib.load('/mnt/data/output/transfer_2217763_files_3b695c8d/BRICK-001/BRICK-001/20220620/MR/eASL_3_delays/005_eASL_3_delays_eASL_3_delays_20220620000000_reala.nii.gz')
loaded_obj = loaded.dataobj
print(loaded_obj.shape)

plt.figure()
f, axarr = plt.subplots(1,4)

axarr[0].imshow(loaded_obj[:,:,20, 0], cmap='gray')
axarr[1].imshow(loaded_obj[:,:,20, 1], cmap='gray')
axarr[2].imshow(loaded_obj[:,:,20, 2], cmap='gray')
axarr[3].imshow(loaded_obj[:,:,20, 3], cmap='gray')

In [ ]:

loaded = nib.load('/mnt/data/festical/001.nii.gz/native_space/arrival.nii.gz')
loaded_obj = loaded.dataobj
print(loaded_obj.shape)

plt.imshow(loaded_obj[:,:,20], cmap='gray')



In [ ]:

loaded = nib.load('/mnt/data/festical/001.nii.gz/native_space/perfusion.nii.gz')
loaded_obj = loaded.dataobj
print(loaded_obj.shape)

plt.imshow(loaded_obj[:,:,20], cmap='gray')



In [ ]:
# files_we_want = glob.glob("/mnt/data/output/transfer_2217763_files_3b695c8d/*/*/*/MR/eASL_3_delays/*.nii.gz")
# files_we_want
files_we_want = glob.glob("/mnt/data/output/transfer_2217763_files_3b695c8d/**/MR/eASL_3_delays/*.nii.gz", recursive=True)

In [ ]:
files_we_want

In [ ]:
len(files_we_want)/2

In [ ]:
image_string= "mnt/data/output/transfer_2217763_files_3b695c8d/BRICK-001/BRICK-001/20220620/MR/eASL_3_delays/005_eASL_3_delays_eASL_3_delays_20220620000000_real.nii.gz"
image_stringa = "/mnt/data/output/transfer_2217763_files_3b695c8d/BRICK-001/BRICK-001/20220620/MR/eASL_3_delays/005_eASL_3_delays_eASL_3_delays_20220620000000_reala.nii.gz"#

In [ ]:
list_of_asl = glob.glob('/mnt/data/only_asl_files/*.nii.gz')
asl_file =widgets.Dropdown(
    options=list_of_asl,
    description='ASL file:')#
#list_of_asl
asl_file

In [ ]:
copied_str = asl_file.value
copied_str

In [ ]:
#iso_str= '/mnt/data/output/transfer_2217763_files_3b695c8d/BRICK-001/BRICK-001/20220620/MR/Isotropic_image/402_Isotropic_image_Isotropic_image_20220620000000.nii.gz'

In [ ]:
#foundation.NiftiSliceViewer(iso_str)

In [ ]:
#iso_img = nib.load(iso_str)

In [ ]:
#iso_img.dataobj.shape

In [ ]:
image = nib.load(image_string)
imagea = nib.load(image_stringa)
loaded = nib.load('/mnt/data/output/transfer_2217763_files_3b695c8d/BRICK-001/BRICK-001/20220620/MR/eASL_3_delays/005_eASL_3_delays_eASL_3_delays_20220620000000_real.nii.gz')
loaded_obj = loaded.dataobj

In [ ]:
image.dataobj.shape

In [ ]:
imagea.dataobj.shape

In [ ]:
#image.header['db_name']

In [ ]:
image

In [ ]:
img_arr = image.dataobj
img_arra = imagea.dataobj

In [ ]:
img_arra.shape

In [ ]:
print(type(img_arr))

In [ ]:
real = np.array(img_arr)
reala = np.array(img_arra)

In [ ]:
list_of_asl = glob.glob('/mnt/data/only_asl_files/*.nii.gz')
asl_file =widgets.Dropdown(
    options=list_of_asl,
    description='ASL file:')#
#list_of_asl
asl_file

In [ ]:
asl_file.value

In [ ]:
loaded.shape

In [ ]:
#plt.imshow(loaded[:,:,20, 3], cmap='gray')

In [ ]:
#real
first = np.array(img_arr)
second = np.array(img_arra) 


loaded = nib.load('/mnt/data/output/transfer_2217763_files_3b695c8d/BRICK-001/BRICK-001/20220620/MR/eASL_3_delays/005_eASL_3_delays_eASL_3_delays_20220620000000_reala.nii.gz')
loaded_obj = loaded.dataobj
print(loaded_obj.shape)

plt.figure()
f, axarr = plt.subplots(1,4)

axarr[0].imshow(loaded_obj[:,:,20, 0], cmap='gray')
axarr[1].imshow(loaded_obj[:,:,20, 1], cmap='gray')
axarr[2].imshow(loaded_obj[:,:,20, 2], cmap='gray')
axarr[3].imshow(loaded_obj[:,:,20, 3], cmap='gray')

In [ ]:

loaded2 = nib.load('/mnt/data/output/transfer_2217763_files_3b695c8d/BRICK-001/BRICK-001/20220620/MR/eASL_3_delays/005_eASL_3_delays_eASL_3_delays_20220620000000_real.nii.gz')
loaded_obj2 = loaded2.dataobj
print(loaded_obj2.shape)


plt.figure()
f, axarr = plt.subplots(1,4)

axarr[0].imshow(loaded_obj2[:,:,20, 0], cmap='gray')
axarr[1].imshow(loaded_obj2[:,:,20, 1], cmap='gray')
#axarr[2].imshow(loaded_obj[:,:,20, 2], cmap='gray')
#axarr[3].imshow(loaded_obj[:,:,20, 3], cmap='gray')

In [ ]:
plt.imshow(loaded_obj2[:,:,20, 1] - loaded_obj2[:,:,20, 0], cmap='grey')

In [ ]:
plt.imshow(second[:,:,20, 0], cmap='grey')

In [ ]:
new_array = first - second
new_array.sum()

In [ ]:
plt.imshow(new_array[:,:,20, 1])
